<a href="https://colab.research.google.com/github/yin00406/Code_GDAL/blob/main/export%20vector%20from%20vector%20collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Iteratively output .shp, .shx, .prj, .dbf for each element in a shapefile**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
root_Drive = "/content/drive/My Drive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from osgeo import ogr
import os

In [5]:
# CONFIG
colName = "PageName"

# Define the path to your original shapefile and the output directory
osr_file = os.path.join(root_Drive, "Research&Project/Cashew/Cashew_map_West-Africa/NGA", "NGA_grids_90km.shp") #c
tst_folder = os.path.join(root_Drive, "Research&Project/Cashew/Cashew_map_West-Africa/NGA/NGA_grids_90km") #c

# Open the original shapefile
driver = ogr.GetDriverByName('ESRI Shapefile')
dataSource = driver.Open(osr_file, 0)  # 0 means read-only. Use 1 for write access.
layer = dataSource.GetLayer()

In [6]:
# Iterate over features in the original shapefile
for feature in layer:
    # Get the value of the 'colName' attribute
    col = feature.GetField(colName)

    # Create a valid filename from the 'colName' attribute
    filename = f"NGA_{col}.shp" #c
    output_shapefile = os.path.join(tst_folder, filename)

    # Check if the output shapefile already exists. If so, delete it
    if os.path.exists(output_shapefile):
        driver.DeleteDataSource(output_shapefile)

    # Create a new shapefile for the current feature
    outDataSource = driver.CreateDataSource(output_shapefile)
    outLayer = outDataSource.CreateLayer("feature", geom_type=layer.GetGeomType())

    # Add fields to the newly created shapefile
    layerDefn = layer.GetLayerDefn()
    for i in range(layerDefn.GetFieldCount()):
        fieldDefn = layerDefn.GetFieldDefn(i)
        outLayer.CreateField(fieldDefn)

    # Create a new feature for the output layer
    outFeature = ogr.Feature(outLayer.GetLayerDefn())

    # Set field values from the original feature
    for i in range(layerDefn.GetFieldCount()):
        outFeature.SetField(outLayer.GetLayerDefn().GetFieldDefn(i).GetNameRef(), feature.GetField(i))

    # Set the geometry
    geom = feature.GetGeometryRef()
    outFeature.SetGeometry(geom.Clone())

    # Add the feature to the output layer
    outLayer.CreateFeature(outFeature)

    # Create the .prj file with the same filename as the shapefile
    prj_filename = output_shapefile.replace(".shp", ".prj")
    # Get the spatial reference from the original layer
    spatialRef = layer.GetSpatialRef()
    # Write the spatial reference to the .prj file
    with open(prj_filename, "w") as prj_file:
        prj_file.write(spatialRef.ExportToWkt())

    # Cleanup
    outFeature = None
    outDataSource = None
    outLayer = None

# Cleanup
dataSource = None